# Decorators

## args and kwargs

In functions

* positional arguments (required)
* keyword arguments (optional, with a default value)

In [38]:
def sum_and_scale(a, b, c):
    return (a + b)*c

In [39]:
sum_and_scale(2, 3, 4)

20

In [40]:
def sum_and_scale(a, b, c=1): # 2 positional 1 required
    return (a + b)*c

In [41]:
sum_and_scale(2, 3)

5

In [42]:
sum_and_scale(2, 3, 4)

20

In [43]:
sum_and_scale(2, 3, c=4)

20

In [44]:
# with any number of positional arguments - collected in tuple args
def sum_and_scale(*args, c=1):
    print(args)
    #return (a + b)*c
    return sum(args)*c

In [45]:
sum_and_scale()

()


0

In [46]:
sum_and_scale(1, 2)

(1, 2)


3

In [47]:
sum_and_scale(1, 2, 3, 4, c=5)

(1, 2, 3, 4)


50

In [48]:
# with any number of keyword arguments - collect in a dict kwargs
def sum_and_scale(*args, **kwargs):
    print(args, kwargs)
    #return (a + b)*c
    #return sum(args)*c
    return sum(args)*kwargs.get('c', 1)

In [49]:
sum_and_scale()

() {}


0

In [50]:
sum_and_scale(1, 2, 3, 4, c=5)

(1, 2, 3, 4) {'c': 5}


50

In [51]:
myargs = (1, 2, 3, 4)
mykwargs = {'c': 5}
sum_and_scale(*myargs, **mykwargs)

(1, 2, 3, 4) {'c': 5}


50

## A timing decorator

In [52]:
import math
import time

In [53]:
def slow(x):
    time.sleep(1)
    return math.sqrt(x)

In [54]:
slow(2)

1.4142135623730951

In [55]:
t1 = time.time()
slow(2)
t2 = time.time()
print("Time spent in slow", t2-t1)

Time spent in slow 1.0027351379394531


In [56]:
def identity(f): # identity decorator
    return f

In [57]:
identity(slow)(2)

1.4142135623730951

In [58]:
def time_me(f):
    def inner(*args, **kwargs):
        t1 = time.time()
        value = f(*args, **kwargs)
        t2 = time.time()
        print(f"Time spent in {f.__name__}", t2-t1)
        return value
    return inner

In [59]:
#time_me(slow)(2)
timed_slow = time_me(slow)
timed_slow(2)

Time spent in slow 1.0050780773162842


1.4142135623730951

In [60]:
# A special syntax
# equivlent to 
# slow = time_me(slow)

@time_me
def slow(x):
    time.sleep(1)
    return math.sqrt(x)

In [61]:
slow(2)

Time spent in slow 1.0051159858703613


1.4142135623730951

## debugging decorator

In [62]:
def debug(f):
    def inner(*args, **kwargs):
        print(f"Input arguments to {f.__name__}", args, kwargs)
        value = f(*args, **kwargs)
        print(f"Return value from {f.__name__}", value)
        return value
    return inner

In [63]:
@debug
def slow(x):
    time.sleep(1)
    return math.sqrt(x)

In [64]:
slow(2)

Input arguments to slow (2,) {}
Return value from slow 1.4142135623730951


1.4142135623730951

## combining decorators

In [66]:
@time_me  # slow = time_me(debug(slow))
@debug
def slow(x):
    time.sleep(1)
    return math.sqrt(x)

In [67]:
slow(2)

Input arguments to slow (2,) {}
Return value from slow 1.4142135623730951
Time spent in inner 1.0049049854278564


1.4142135623730951

In [68]:
@debug
@time_me 
def slow(x):
    time.sleep(1)
    return math.sqrt(x)

In [69]:
slow(2)

Input arguments to inner (2,) {}
Time spent in slow 1.0006318092346191
Return value from inner 1.4142135623730951


1.4142135623730951

In [70]:
from functools import wraps # to get correct function names

def time_me(f):
    @wraps(f)
    def inner(*args, **kwargs):
        t1 = time.time()
        value = f(*args, **kwargs)
        t2 = time.time()
        print(f"Time spent in {f.__name__}", t2-t1)
        return value
    return inner
    
def debug(f):
    @wraps(f)
    def inner(*args, **kwargs):
        print(f"Input arguments to {f.__name__}", args, kwargs)
        value = f(*args, **kwargs)
        print(f"Return value from {f.__name__}", value)
        return value
    return inner

In [71]:
@debug
@time_me 
def slow(x):
    time.sleep(1)
    return math.sqrt(x)

In [72]:
slow(2)

Input arguments to slow (2,) {}
Time spent in slow 1.0050640106201172
Return value from slow 1.4142135623730951


1.4142135623730951

## time_me as context manager

In [79]:
class TimeMe:
    def __init__(self):
        ...

    def __enter__(self):
        self.t1 = time.time()

    def __exit__(self, *args):
        self.t2 = time.time()
        print('Time used', self.t2 - self.t1) 

In [81]:
def slow(x):
    time.sleep(1)
    return math.sqrt(x)

with TimeMe():
    value = slow(2)

print(value)

Time used 1.0031938552856445
1.4142135623730951
